In [33]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import json as js
import os
import re

In [23]:
df = pd.read_csv(r"C:\Users\pipoc\Documents\Data_Felipe\UFAPE\RNN - FM\atividade1-parte1\data\processed\dallas_games_2024-25.csv")
df2 = pd.read_csv(r"C:\Users\pipoc\Documents\Data_Felipe\UFAPE\RNN - FM\atividade1-parte1\data\processed\dallas_players_2024-25.csv")

In [24]:
df.head(5)

,data-jogo,resultado,mando-de-jogo,pontos,saldo-pontos,arremessos-convertidos,arremessos-tentados,porcentagem-arremessos,triplos-convertidos,triplos-tentados,...,lances-livres-tentados,porcentagem-lances-livres,rebotes-ofensivos,rebotes-defensivos,rebotes-totais,assistencias,roubos,tocos,erros,faltas
0,20241007,0,1,116,-5.0,41,90,0.456,13,33,...,32,0.656,7,29,36,26,19,3,14,26
1,20241010,0,1,102,-5.0,39,87,0.448,8,26,...,21,0.762,10,35,45,21,8,10,17,27
2,20241014,0,0,96,-14.0,35,84,0.417,15,40,...,16,0.688,8,30,38,21,6,8,29,20
3,20241017,1,1,109,25.0,40,80,0.500,8,28,...,25,0.840,9,44,53,23,8,9,14,10
4,20241024,1,1,120,11.0,42,102,0.412,19,44,...,23,0.739,17,33,50,27,9,5,9,21


In [25]:
print(df['resultado'].value_counts())

resultado
0    47
1    41
Name: count, dtype: int64


In [26]:
df1_numerico = df.select_dtypes(include=np.number)

print("="*60)
print("CORRELAÇÃO DAS VARIÁVEIS COM 'RESULTADO' (df)")
print("="*60)

correlacoes_resultado = df1_numerico.corr()['resultado'].sort_values(ascending=False)

variaveis_fortes_logistica = correlacoes_resultado.drop(
    ['resultado', 'saldo_pontos'],
    errors='ignore'
).head(10)

print(variaveis_fortes_logistica)

CORRELAÇÃO DAS VARIÁVEIS COM 'RESULTADO' (df)
saldo-pontos              0.783287
pontos                    0.493078
porcentagem-triplos       0.469378
triplos-convertidos       0.409162
rebotes-totais            0.406022
porcentagem-arremessos    0.380350
arremessos-convertidos    0.377910
rebotes-defensivos        0.375177
assistencias              0.343920
roubos                    0.257512
Name: resultado, dtype: float64


In [27]:
df.columns = df.columns.str.replace('-', '_')

print("Colunas do df1 renomeadas para usar underscores (_).")

Colunas do df1 renomeadas para usar underscores (_).


In [28]:
df

,data_jogo,resultado,mando_de_jogo,pontos,saldo_pontos,arremessos_convertidos,arremessos_tentados,porcentagem_arremessos,triplos_convertidos,triplos_tentados,...,lances_livres_tentados,porcentagem_lances_livres,rebotes_ofensivos,rebotes_defensivos,rebotes_totais,assistencias,roubos,tocos,erros,faltas
0,20241007,0,1,116,-5.0,41,90,0.456,13,33,...,32,0.656,7,29,36,26,19,3,14,26
1,20241010,0,1,102,-5.0,39,87,0.448,8,26,...,21,0.762,10,35,45,21,8,10,17,27
2,20241014,0,0,96,-14.0,35,84,0.417,15,40,...,16,0.688,8,30,38,21,6,8,29,20
3,20241017,1,1,109,25.0,40,80,0.500,8,28,...,25,0.840,9,44,53,23,8,9,14,10
4,20241024,1,1,120,11.0,42,102,0.412,19,44,...,23,0.739,17,33,50,27,9,5,9,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,20250409,0,1,97,-15.0,36,72,0.500,9,26,...,23,0.696,9,29,38,30,6,6,19,14
84,20250411,1,1,124,22.0,49,89,0.551,15,34,...,17,0.647,12,44,56,34,8,10,17,12
85,20250413,0,0,97,-35.0,38,87,0.437,5,28,...,22,0.727,10,28,38,20,6,3,18,13
86,20250416,1,0,120,14.0,41,83,0.494,14,28,...,26,0.923,10,31,41,29,9,8,9,15


In [ ]:
formula_logistica_limpa = (
    "resultado ~ porcentagem_triplos + rebotes_totais + mando_de_jogo"
)

modelo_logistico = smf.logit(formula_logistica_limpa, data=df).fit()

print("="*80)
print("🎯 RESULTADOS DA REGRESSÃO LOGÍSTICA (Probabilidade de Vitória)")
print("="*80)
print(modelo_logistico.summary())

Optimization terminated successfully.
         Current function value: 0.444660
         Iterations 7
🎯 RESULTADOS DA REGRESSÃO LOGÍSTICA (Probabilidade de Vitória)
                           Logit Regression Results                           
Dep. Variable:              resultado   No. Observations:                   88
Model:                          Logit   Df Residuals:                       84
Method:                           MLE   Df Model:                            3
Date:                Mon, 03 Nov 2025   Pseudo R-squ.:                  0.3563
Time:                        15:32:59   Log-Likelihood:                -39.130
converged:                       True   LL-Null:                       -60.792
Covariance Type:            nonrobust   LLR p-value:                 2.100e-09
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -16.8

In [50]:
import json
import os
import re
import statsmodels.formula.api as smf
import pandas as pd

def salvar_modelo_json_final_v6(modelo, nome_modelo):
    """
    Versão FINAL e MAIS ROBUSTA: Extração de todas as métricas por lista de pares, 
    eliminando o KeyError: 1 e a dependência de colunas numeradas.
    """
    
    # 1. Extração de Coeficientes (Sem alterações, já está funcionando)
    resultados_coef = modelo.summary2().tables[1] 
    coeficientes_data = []

    # Detecta a coluna de P-valor ('P>|z|' para Logit ou 'P>|t|' para OLS)
    if 'P>|z|' in resultados_coef.columns:
        p_valor_key = 'P>|z|'
    elif 'P>|t|' in resultados_coef.columns:
        p_valor_key = 'P>|t|'
    else:
        raise KeyError("Coluna de p-valor ('P>|z|' ou 'P>|t|') não encontrada no summary.")
        
    for nome, coef, p_valor in zip(
        resultados_coef.index, 
        resultados_coef['Coef.'], 
        resultados_coef[p_valor_key] 
    ):
        coeficientes_data.append({
            'variavel': nome,
            'coeficiente': round(coef, 4),
            'p_valor': round(p_valor, 4),
            'significativo': p_valor < 0.05
        })

    # 2. Extração de Métricas de Força (Leitura de Pares Robustos)
    
    # Normaliza o texto e achata para buscar as chaves.
    sumario_texto = modelo.summary().as_text()
    texto_normalizado = " ".join(sumario_texto.split())
    
    # Regex para achar as métricas de interesse e seus valores (agnóstico à linha)
    metrics_map = {}
    
    # Busca Pseudo R-squ. e LLR p-value (Logit)
    pseudo_r2 = re.search(r'Pseudo R-squ.: ([\d.e-]+)', texto_normalizado)
    llr_p = re.search(r'LLR p-value: ([\d.e-]+)', texto_normalizado)
    
    # Busca R-squared e Prob (F-statistic) (OLS)
    r2_ols = re.search(r'R-squared: ([\d.]+)', texto_normalizado)
    prob_f_ols = re.search(r'Prob \(F-statistic\): ([\d.e-]+)', texto_normalizado)
    
    # Mapeamento
    if pseudo_r2:
        metrics_map['R_squared'] = float(pseudo_r2.group(1))
        metrics_map['F_Prob'] = float(llr_p.group(1)) if llr_p else 0.0
    elif r2_ols:
        metrics_map['R_squared'] = float(r2_ols.group(1))
        metrics_map['F_Prob'] = float(prob_f_ols.group(1)) if prob_f_ols else 0.0
        
    
    # 3. Montagem do JSON
    dados_json = {
        'nome_do_modelo': nome_modelo,
        'variavel_dependente': modelo.model.endog_names,
        'R_squared': metrics_map.get('R_squared', 0.0),
        'Adj_R_squared': metrics_map.get('Adj_R_squared', 0.0), # Deixamos 0.0, pois o Logit não tem
        'F_Prob': metrics_map.get('F_Prob', 0.0),
        'equacao_coeficientes': coeficientes_data
    }

    # 4. Salvamento no Diretório (Mantido)
    DIRETORIO_DESTINO = './data/colabs/json/'
    os.makedirs(DIRETORIO_DESTINO, exist_ok=True)
    
    nome_arquivo_formatado = nome_modelo.lower().replace(' ', '_')
    caminho_arquivo = os.path.join(DIRETORIO_DESTINO, f"{nome_arquivo_formatado}.json")
    
    with open(caminho_arquivo, 'w', encoding='utf-8') as f:
        json.dump(dados_json, f, ensure_ascii=False, indent=4)
        
    print(f"\n✅ JSON '{nome_modelo}' salvo com sucesso em: {caminho_arquivo}")
    
    return dados_json

In [51]:

formula_logistica_limpa = "resultado ~ porcentagem_triplos + rebotes_totais + mando_de_jogo"
modelo_logistico_final = smf.logit(formula_logistica_limpa, data=df).fit()

# Usando a função de salvamento validada
json_modelo_logistico = salvar_modelo_json_final_v6(
    modelo_logistico_final, 
    "Probabilidade_Vitoria"
)

print(json.dumps(json_modelo_logistico, indent=4))

Optimization terminated successfully.
         Current function value: 0.444660
         Iterations 7

✅ JSON 'Probabilidade_Vitoria' salvo com sucesso em: ./data/colabs/json/probabilidade_vitoria.json
{
    "nome_do_modelo": "Probabilidade_Vitoria",
    "variavel_dependente": "resultado",
    "R_squared": 0.3563,
    "Adj_R_squared": 0.0,
    "F_Prob": 2.1e-09,
    "equacao_coeficientes": [
        {
            "variavel": "Intercept",
            "coeficiente": -16.8054,
            "p_valor": 0.0,
            "significativo": true
        },
        {
            "variavel": "porcentagem_triplos",
            "coeficiente": 21.5743,
            "p_valor": 0.0001,
            "significativo": true
        },
        {
            "variavel": "rebotes_totais",
            "coeficiente": 0.1959,
            "p_valor": 0.0002,
            "significativo": true
        },
        {
            "variavel": "mando_de_jogo",
            "coeficiente": 0.7212,
            "p_valor": 0.2104,

In [ ]:

Dep. Variable:              resultado   No. Observations:                   88
Model:                          Logit   Df Residuals:                       84
Method:                           MLE   Df Model:                            3
Date:                Mon, 03 Nov 2025   Pseudo R-squ.:                  0.3563
Time:                        15:32:59   Log-Likelihood:                -39.130
converged:                       True   LL-Null:                       -60.792
Covariance Type:            nonrobust   LLR p-value:                 2.100e-09
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -16.8054      3.778     -4.448      0.000     -24.210      -9.401
porcentagem_triplos    21.5743      5.504      3.919      0.000      10.786      32.363
rebotes_totais          0.1959      0.053      3.680      0.000       0.092       0.300
mando_de_jogo           0.7212      0.576      1.253      0.210      -0.407       1.850
=======================================================================================